In [11]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt                
from sklearn.preprocessing import LabelEncoder
from src.wrapped import Wrapped
from src.analysesV02 import Analytics 
from src.trainV02 import TrainModels
from skopt.space import Real, Integer, Categorical
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier
from xgboost import XGBClassifier

# Importando dados para treinamento

In [12]:
ac = Analytics()
tm = TrainModels()
wp = Wrapped(
    '../data/row/',
    '../data/processed/',
    '../data/files/'
)

# dataframe
df = wp.load_data('df_instrumentos_features_selecionadas').drop(columns=['file_name'])
df = df.query("instrumento != 'voice' & instrumento != 'synthesizer'")

# dataframes por instrumentos
inst_corda    = ["cello", "guitar", "violin", "bass", "banjo", "mandolin", "ukulele"]
inst_percusao = ["mallet_percussion", "drums", "cymbals"]
inst_sopro    = ["clarinet", "trombone", "flute", "trumpet", "saxophone"]
inst_aerofone = ["accordion", "organ", "piano"] 

df_inst_aerofone = df[df['instrumento'].isin(inst_aerofone)]
df_inst_sopro    = df[df['instrumento'].isin(inst_sopro)]
df_inst_corda    = df[df['instrumento'].isin(inst_corda)]
df_inst_percusao = df[df['instrumento'].isin(inst_percusao)]

# Pre-processamento

- Label Encoder

In [13]:
le = LabelEncoder()

df_inst_aerofone['labels'] = le.fit_transform(df_inst_aerofone.instrumento)
df_inst_sopro['labels']    = le.fit_transform(df_inst_sopro.instrumento)
df_inst_corda['labels']    = le.fit_transform(df_inst_corda.instrumento)
df_inst_percusao['labels'] = le.fit_transform(df_inst_percusao.instrumento)

# Analise de Anomalias

#### Tunning de Hiperparametros

- XGBoost

In [14]:
parametros = {
    "eta": Real(0.01, 0.2),
    "max_depth": Integer(6, 20),
    "gamma":  Integer(0, 10),
    "learning_rate": Real(0,1),
    "subsample": Real(0.5, 1)
}

tm.train_tunning_hyperparameters(
    dataframe=df_inst_corda, 
    model=XGBClassifier(),
    parameters=parametros, 
    filename="resultados_parametros_corda_XGBoost"
)

tm.train_tunning_hyperparameters(
    dataframe=df_inst_aerofone, 
    model=XGBClassifier(),
    parameters=parametros, 
    filename="resultados_parametros_aerofone_XGBoost"
)

tm.train_tunning_hyperparameters(
    dataframe=df_inst_percusao, 
    model=XGBClassifier(),
    parameters=parametros, 
    filename="resultados_parametros_percusao_XGBoost"
)

tm.train_tunning_hyperparameters(
    dataframe=df_inst_sopro, 
    model=XGBClassifier(),
    parameters=parametros, 
    filename="resultados_parametros_sopro_XGBoost"
)

interação 0 -> Metric: euclidean, Algoritmo: auto, neighbor: 3
interação 0 - Acuracy models: 53.86559700627556
interação 1 -> Metric: manhattan, Algoritmo: auto, neighbor: 3
interação 1 - Acuracy models: 54.35365025748018
interação 2 -> Metric: minkowski, Algoritmo: auto, neighbor: 3
interação 2 - Acuracy models: 52.76737370225027
interação 3 -> Metric: euclidean, Algoritmo: ball_tree, neighbor: 3
interação 3 - Acuracy models: 52.72564701207093
interação 4 -> Metric: manhattan, Algoritmo: ball_tree, neighbor: 3
interação 4 - Acuracy models: 53.702333051843766
interação 5 -> Metric: minkowski, Algoritmo: ball_tree, neighbor: 3
interação 5 - Acuracy models: 52.88982166807411
interação 6 -> Metric: euclidean, Algoritmo: kd_tree, neighbor: 3
interação 6 - Acuracy models: 54.272597817627876
interação 7 -> Metric: manhattan, Algoritmo: kd_tree, neighbor: 3
interação 7 - Acuracy models: 53.946815028231754
interação 8 -> Metric: minkowski, Algoritmo: kd_tree, neighbor: 3
interação 8 - Acuracy 

- HistGBM

In [ ]:
parametros_histGB = {
    "min_samples_leaf": Integer(5, 20),
    "max_depth": Integer(6, 20),
    "loss": Categorical(['log_loss','auto','categorical_crossentropy']), 
    "max_bins": Integer(100, 250)
}

print("instrumeno de percursão")
tm.train_tunning_hyperparameters(
    dataframe=df_inst_percusao, 
    model=HistGradientBoostingClassifier(), 
    parameters=parametros_histGB, 
    filename="resultados_parametros_percusao_histGB"
)

print("instrumeno de corda")
tm.train_tunning_hyperparameters(
    dataframe=df_inst_corda, 
    model=HistGradientBoostingClassifier(), 
    parameters=parametros_histGB, 
    filename="resultados_parametros_corda_histGB"
)

print("instrumeno de sopro")
tm.train_tunning_hyperparameters(
    dataframe=df_inst_sopro, 
    model=HistGradientBoostingClassifier(), 
    parameters=parametros_histGB, 
    filename="resultados_parametros_sopro_histGB"
)

print("instrumeno de aerofone")
tm.train_tunning_hyperparameters(
    dataframe=df_inst_aerofone, 
    model=HistGradientBoostingClassifier(), 
    parameters=parametros_histGB, 
    filename="resultados_parametros_aerofone_histGB"
)

- Random Forest

In [ ]:
 Random Forest 
parametros_rf = {
    "criterion": Categorical(['gini','entropy']),
    "max_depth": Integer(6, 20),
    "min_samples_split": Integer(2, 10),
    "min_samples_leaf": Integer(2, 10),
    "max_features": Categorical(['auto', 'sqrt','log2']), 
    "bootstrap": Categorical([True, False]),
    "n_estimators": Integer(100, 500)
}

tm.train_tunning_hyperparameters(
    dataframe=df_inst_aerofone, 
    model=RandomForestClassifier(), 
    parameters=parametros_rf, 
    filename="resultados_parametros_aerofone_random_forest"
)

tm.train_tunning_hyperparameters(
    dataframe=df_inst_sopro, 
    model=RandomForestClassifier(), 
    parameters=parametros_rf, 
    filename="resultados_parametros_sopro_random_forest"
)

tm.train_tunning_hyperparameters(
    dataframe=df_inst_percusao, 
    model=RandomForestClassifier(), 
    parameters=parametros_rf, 
    filename="resultados_parametros_percusao_random_forest"
)

tm.train_tunning_hyperparameters(
    dataframe=df_inst_corda, 
    model=RandomForestClassifier(), 
    parameters=parametros_rf, 
    filename="resultados_parametros_corda_random_forest"
)